In [6]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [7]:
# Tool - 1 , Here out 1st Tool is Wikipedia 
api = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki_tool = WikipediaQueryRun(api_wrapper=api)


In [3]:
pip install wikipedia

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11785 sha256=c725ac20d46b8070700027dbf98061a182d22be1b6e0fa5cc7d332213a5081a7
  Stored in directory: c:\users\nalla\appdata\local\pip\cache\wheels\5e\b6\c5\93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [8]:
wiki_tool.name

'wikipedia'

In [9]:
## Libraries needed to fetch the data from URL
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
loader_AA_TC = WebBaseLoader("https://www.aa.com/i18n/customer-service/support/legal-information.jsp")
AA_TC_data = loader_AA_TC.load()
AA_TC_fecthed_data = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(AA_TC_data)
vectordb_AA_TC = FAISS.from_documents(AA_TC_fecthed_data[:20],OllamaEmbeddings())
AA_TC_retriver = vectordb_AA_TC.as_retriever()
AA_TC_retriver

C:\Users\nalla\AppData\Local\Temp\ipykernel_22060\2380442911.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  vectordb_AA_TC = FAISS.from_documents(AA_TC_fecthed_data[:20],OllamaEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023A4A9E6710>, search_kwargs={})

In [11]:
loader_AA_CC = WebBaseLoader("https://www.aa.com/i18n/customer-service/support/conditions-of-carriage.jsp")
AA_CC_data = loader_AA_CC.load()
AA_CC_fetched_data = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(AA_CC_data)
vectordb_AA_CC = FAISS.from_documents(AA_CC_fetched_data[:20],OllamaEmbeddings())
AA_CC_retriver = vectordb_AA_CC.as_retriever()
AA_CC_retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023A1B2272B0>, search_kwargs={})

In [12]:
tools = [wiki_tool,AA_TC_retriver,AA_CC_retriver]

In [13]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\nalla\\anaconda3\\envs\\myenv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023A4A9E6710>, search_kwargs={}),
 VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023A1B2272B0>, search_kwargs={})]

In [15]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama2")
llm

OllamaLLM(model='llama2')

In [53]:
question = "Can I checkin my baggage two days before the flight go have fun around the city for 2 days and finally board the flight"


routing_prompt = f"""
You're an AI assistant with access to 3 tools:

1. "aa_cc" - Use this for questions about American Airlines baggage.
2. "aa_tc" - Use this for questions about American Airlines refund policies.


Based on the question below, return only the tool name (wiki / aa_tc / aa_cc).

Question: {question}
Tool: """

tool_choice = llm.invoke(routing_prompt).strip().lower()
print(tool_choice)

if "aa_cc" in tool_choice:
    docs = AA_TC_retriver.get_relevant_documents(question)
    context = "\n\n".join(doc.page_content for doc in docs)
elif "aa_tc" in tool_choice:
    docs = AA_CC_retriver.get_relevant_documents(question)
    context = "\n\n".join(doc.page_content for doc in docs)
else:
    context = wiki_tool.run(question)



aa_cc


In [ ]:
## This is an optional code to show how to import predefined prompts that arae already in a structured and are present in lanchain hub
from langchain import hub

prompt_demo = hub.pull("hwchase17/openai-functions-agent")
prompt_demo.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [55]:
# === Step 3: Ask the final LLM with context ===
final_prompt = f"""
You are a helpful customer service assistant trained on American Airlines policies.

Step 1: but if you are answering from the <context> tell the cusomter confidentely I have checked the Terms and conditons for you and this is what I found
Step 2: Try to find the answer in the provided context. Quote relevant policy lines when possible never tell customer to go check this part for more info.
Step 3: If the answer is NOT found in the context, use your general airline knowledge, but mention that you're doing so 

<context>
{context}
</context>

Question: {question}


Answer:"""

answer = llm.invoke(final_prompt)
print(answer)



answer = llm.invoke(final_prompt)


Thank you for reaching out to us! I'm happy to help you with your question. However, I need to inform you that according to American Airlines' policies, check-in for flights is typically available 24 hours before the scheduled departure time, and baggage drop-off is usually available 30 minutes before the flight's scheduled departure time.

It's important to note that checking in for a flight too early can lead to issues with baggage delivery and boarding, so it's best to check in at the appropriate time. Additionally, American Airlines does not allow passengers to book multiple flights under different names or aliases, so you will need to use your real name and accurate contact information when booking a flight.

If you have any other questions or concerns, please feel free to ask!
